# Day 19

https://adventofcode.com/2022/day/19

In [1]:
import re
import numpy as np

def parse19(filename):
    with open(filename) as f:
        data = [ [ int(i) for i in re.findall("\d+",l) ] for l in f.readlines() ]
        # saving robot costs as array of resources [ore, clay, obsidian, geodes]
        bps = []
        for d in data:
            ore_rob_cost = np.array([d[1], 0, 0, 0])
            cla_rob_cost = np.array([d[2], 0, 0, 0])
            obs_rob_cost = np.array([d[3], d[4], 0, 0])
            geo_rob_cost = np.array([d[5], 0, d[6], 0])
            bps.append( [ore_rob_cost, cla_rob_cost, obs_rob_cost, geo_rob_cost] )
        return bps

bps = parse19("examples/example19.txt")
bps

[[array([4, 0, 0, 0]),
  array([2, 0, 0, 0]),
  array([ 3, 14,  0,  0]),
  array([2, 0, 7, 0])],
 [array([2, 0, 0, 0]),
  array([3, 0, 0, 0]),
  array([3, 8, 0, 0]),
  array([ 3,  0, 12,  0])]]

In [8]:
from queue import Queue

def hash_state(state):
    time,robots,resources = state
    h = str(time)+"_"
    for r in robots:
        h += "_"+str(r)
    h += "_"
    for r in resources:
        h += "_"+str(r)   
    return(h)

def max_geodes(bp,timemax=24):

    # compute maximum amount of each resource needed to build any robot
    max_res = np.zeros(4,dtype=int)
    for cost in bp:
        for j in range(len(cost)):
            if cost[j]>max_res[j]:
                max_res[j]=cost[j]

    # initial state
    robots    = np.array([1, 0, 0, 0])
    resources = np.array([0, 0, 0, 0])
    time  = 0
    start = (time,robots,resources)
    
    # BFS-like search of state evolutions
    states = { hash_state(start) }
    geodes_max = 0
    q = Queue()
    q.put(start)
    
    while not q.empty():
        
        # get state
        time, robots, resources = q.get()
        
        # fast-forward in time to a state where a new robot can be built
        # given existing resources and what could be built in the interval       
        for i in range(len(bp)): # index of robot to be built
            cost = bp[i]
            time_needed = [ 0, 0, 0, 0 ] # time needed to gather resources to build robot
            for j in range(len(cost)): # needed resources
                if cost[j]:
                    if cost[j]<=resources[j]: # state already has enough of this resource to build robot
                        continue
                    else: # compute time needed to produce resource
                        if robots[j]: # have robot(s) to produce resource
                            time_needed[j] = (cost[j] - resources[j]) // robots[j] + int((cost[j] - resources[j])%robots[j]>0)
                        else: # no robot to build resource, storing a large time to reject construction
                            time_needed[j] = timemax+1
            dt = max(time_needed) # choosing time from most time-consuming resource
            if time+dt+1+1<=timemax: # resources can be gathered and robot can be built in available time 
                                     # and new robot will have time to do something (+1 minute), otherwise useless
                # collect resources with initially available robots, spend to build new robot
                resources_new = resources + (dt+1)*robots - cost
                # build new robot
                robots_new = np.copy(robots)
                robots_new[i] += 1
                
                # OPTIMISATIONS: The code converges to the correct results even without these optimisations, 
                # but the search space becomes bery large and the execution time very slow...
                
                # 1) If it takes N resources to build a robot, it's usess to have M>N robots collecting that resource
                # so I can speed-up the process by avoiding to re-enque states with too many useless robots. 
                # This is enough to solve Part 1 in a decent time
                if not ( robots_new <= max_res )[:3].all() : # do not consider geodes 
                    continue
                    
                # 2) Let's imagine that from next round only geodes robots will be added to this state (regardless
                # of whether this is possible in term of resources). If even in this overoptimistic conditions the
                # state cannot produce more geodes than the current maximum, it's useless to re-enque this state
                timeleft = timemax - (time+dt+1)
                geodes_new_ideal = (timeleft-1)*(timeleft)//2 # triangular number for timeleft-1, since geodes
                                                              # built at last minute cannot build anything
                geodes_final_ideal = resources_new[3] + timeleft*robots_new[3] + geodes_new_ideal
                if geodes_final_ideal<=geodes_max:
                    continue

                # re-enque new state
                state_new = (time+dt+1,robots_new,resources_new)
                h = hash_state(state_new)
                if h not in states:
                    q.put(state_new)
                    states.add(h)

        # compute geodes from current state
        geodes_this_state = resources[3] + robots[3]*(timemax-time)
        if geodes_this_state > geodes_max:
            geodes_max = geodes_this_state
    
    return geodes_max

In [9]:
def part1(filename):
    bps = parse19(filename)
    q = 0
    print("| Blueprint | Geodes (24) | Quality |")
    print("|-----------+-------------+---------|")
    for i,bp in enumerate(bps):
        g = max_geodes(bp,24)
        print("| {:9d} | {:11d} | {:7d} | ".format(i+1,g,(i+1)*g))
        q += (i+1)*g
    print("\n Sum quality levels: {}".format(q))
    return q

In [10]:
test1 = part1("examples/example19.txt") # 33

| Blueprint | Geodes (24) | Quality |
|-----------+-------------+---------|
|         1 |           9 |       9 | 
|         2 |          12 |      24 | 

 Sum quality levels: 33


In [11]:
import time
tic = time.perf_counter()

part1 = part1("AOC2022inputs/input19.txt") # 1659

toc = time.perf_counter()
print("\nRunning time = {} s".format(int(toc-tic)))

| Blueprint | Geodes (24) | Quality |
|-----------+-------------+---------|
|         1 |           0 |       0 | 
|         2 |           2 |       4 | 
|         3 |           2 |       6 | 
|         4 |           2 |       8 | 
|         5 |           6 |      30 | 
|         6 |           4 |      24 | 
|         7 |          13 |      91 | 
|         8 |           5 |      40 | 
|         9 |           0 |       0 | 
|        10 |           0 |       0 | 
|        11 |           1 |      11 | 
|        12 |           1 |      12 | 
|        13 |           3 |      39 | 
|        14 |           4 |      56 | 
|        15 |           5 |      75 | 
|        16 |           1 |      16 | 
|        17 |           2 |      34 | 
|        18 |           3 |      54 | 
|        19 |           1 |      19 | 
|        20 |           9 |     180 | 
|        21 |           0 |       0 | 
|        22 |           1 |      22 | 
|        23 |           1 |      23 | 
|        24 |           7 |

In [12]:
def part2(filename):
    bps = parse19(filename)
    p = 1
    print("| Blueprint | Geodes (32) |")
    print("|-----------+-------------|")
    for i,bp in enumerate(bps):
        g = max_geodes(bp,32)
        print("| {:9d} | {:11d} | ".format(i+1,g))
        p *= g
        if i+1==3:
            break
    print("\n Product of max geodes: {}".format(p))
    return p

In [13]:
test2 = part2("examples/example19.txt") # 56 * 62 = 3472

| Blueprint | Geodes (32) |
|-----------+-------------|
|         1 |          56 | 
|         2 |          62 | 

 Product of max geodes: 3472


In [14]:
tic = time.perf_counter()

part2 = part2("AOC2022inputs/input19.txt") # 6804

toc = time.perf_counter()
print("\nRunning time = {} s".format(int(toc-tic)))

| Blueprint | Geodes (32) |
|-----------+-------------|
|         1 |           9 | 
|         2 |          27 | 
|         3 |          28 | 

 Product of max geodes: 6804

Running time = 23 s
